In [1]:
import pandas as pd
import numpy as np
train=pd.read_csv('../data/train_raw.csv')
test=pd.read_csv('../data/test_raw.csv')
train.head(2)

,id,질문_1,질문_2,category,답변_1,답변_2,답변_3,답변_4,답변_5
0,TRAIN_000,면진장치가 뭐야?,면진장치에 사용되는 주요 기술은 무엇인가요?,건축구조,면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 ...,"면진장치란 건물의 지반에서 발생하는 진동 에너지를 흡수하여 건물을 보호하고, 진동을...",면진장치란 지반으로부터 발생하는 진동 에너지를 흡수하여 건물에 전달되는 진동을 줄여...,면진장치는 건물의 지반으로부터 오는 진동 에너지를 흡수하여 건물에 전달되는 진동을 ...,면진장치는 건물에 오는 지반 진동의 영향을 최대한으로 흡수하여 건물에 전달되는 진동...
1,TRAIN_001,내진설계의 종류 좀 알려줘,내진설계에는 어떤 종류가 있는지 자세히 알려주실 수 있나요?,건축구조,"내진 설계의 종류로 내진구조, 제진구조, 면진구조가 있습니다.","내진설계에는 내진구조, 제진구조, 면진구조가 있습니다. 내진구조는 건물 구조물이 지...","내진설계에는 주로 내진구조, 제진구조, 면진구조의 세 가지 종류가 있습니다. 이들은...","내진설계에는 주로 내진구조, 제진구조, 면진구조가 사용됩니다. 내진구조는 건물 구조...","내진 설계에는 다양한 종류가 있지만, 대표적으로 내진구조, 제진구조, 면진구조가 있..."


In [5]:
category_list = []

for i in range(len(train)):
    row = train.iloc[i]
    category_list.append(row['category'])

category_list = list(set(category_list))
category_list

['건축구조', '시공', '기타', '인테리어', '마감하자', '타 마감하자', '마감재']

In [16]:
train_dict = {}
for i in range(len(train)):
    row = train.iloc[i]
    train_dict[row['id']] = {
        'q1':row['질문_1'],
        'q2':row['질문_2'],
        
        'a1':row['답변_1'],
        'a2':row['답변_2'],
        'a3':row['답변_3'],
        'a4':row['답변_4'],
        'a5':row['답변_5'],
    }

In [18]:
import random
import itertools

# 같은 카테고리 안에서 질문 2개&그에 따른 답변 2개씩 조합
setlist = []

random.seed(0)
qs = ['질문_1', '질문_2']
rs = ['답변_1', '답변_2', '답변_3', '답변_4', '답변_5']

# 1. 카테고리별로 질문 2개씩 조합
for category in category_list:
    category_idx = train[train['category'] == category]['id']
    # 2. row 2개씩 조합
    question_combinations = list(itertools.combinations(category_idx, 2))
    
    lens = len(question_combinations)

    print('*'*30)
    print(category, '조합 :', lens)

    for i, j in question_combinations:
        item1 =train_dict[i]
        item2 =train_dict[j]

        q1_1 = item1['q1']
        q1_2 = item1['q2']
        q2_1 = item2['q1']
        q2_2 = item2['q2']

        q1_list = [q1_1, q1_2]
        q2_list = [q2_1, q2_1]

        a1_1 = item1['a1']
        a1_2 = item1['a2']
        a1_3 = item1['a3']
        a1_4 = item1['a4']
        a1_5 = item1['a5']

        a2_1 = item2['a1']
        a2_2 = item2['a2']
        a2_3 = item2['a3']
        a2_4 = item2['a4']
        a2_5 = item2['a5']

        a1_list = [a1_1, a1_2, a1_3, a1_4, a1_5]
        a2_list = [a2_1, a2_2, a2_3, a2_4, a2_5]

        for q1, q2 in itertools.product(q1_list, q2_list):
            for a1, a2 in itertools.product(a1_list, a2_list):
                question = f'{q1} {q2}'
                answer = f'{a1} {a2}'
                setlist.append(pd.DataFrame({
                    '질문':[question],
                    '답변':[answer],
                    'category':[category]
                }))
    

******************************
건축구조 조합 : 465
******************************
시공 조합 : 6105
******************************
기타 조합 : 351
******************************
인테리어 조합 : 7503
******************************
마감하자 조합 : 1770
******************************
타 마감하자 조합 : 190
******************************
마감재 조합 : 36856


In [23]:
from tqdm import tqdm

setlist_plain = []
for _, row in tqdm(train.iterrows()):
    for q_col in ['질문_1', '질문_2']:
        for a_col in ['답변_1', '답변_2', '답변_3', '답변_4', '답변_5']:
            setlist_plain.append(pd.DataFrame({'질문': [row[q_col]], '답변': [row[a_col]], 'category': [row['category']]}))

data_list = setlist + setlist_plain
data_small_list = setlist[:200000] + setlist_plain

random.shuffle(data_list)
random.shuffle(data_small_list)

644it [00:00, 781.77it/s]


In [25]:
print(len(setlist), len(setlist_plain))
print(len(data_small_list))

5324000 6440
206440


In [26]:
#df = pd.concat(data_list).reset_index(drop=True)
df_small = pd.concat(data_small_list).reset_index(drop=True)

In [27]:
df_small['category'].value_counts()

category
시공        154610
건축구조       46810
마감재         2720
인테리어        1230
마감하자         600
기타           270
타 마감하자       200
Name: count, dtype: int64

In [31]:
df_small = df_small.drop('category', axis=1)

In [32]:
from sklearn.model_selection import train_test_split
train_ratio = 0.9 
train_df, dev_df = train_test_split(df_small, test_size=1-train_ratio, random_state=42)
print(train_df.shape)

(185796, 2)


In [33]:
train_df.to_csv('../data/train_v2.csv')
dev_df.to_csv('../data/eval_v2.csv')